## Busca Vetorial com Elasticsearch

Uma das novas funcionalidades do **Elasticsearch** é a capacidade de operar com **vetores**, funcionando como um **banco vetorial**. Essa funcionalidade ganhou destaque com o aumento da utilização de **transformers**, redes neurais capazes de representar um texto como um vetor numérico.

Essa nova forma de representar textos permite identificar conteúdos com **significados semelhantes**, mesmo quando são utilizadas **palavras diferentes** — algo que algoritmos tradicionais, como **TF-IDF** e **Okapi BM25**, não conseguem fazer com a mesma eficiência.

Neste notebook, iremos:
- **Salvar os vetores das frases**, e
- **Realizar buscas utilizando o vetor da frase de consulta**.

Para gerar os vetores das frases, será utilizado  o modelo [BAAI/bge-m3](https://huggingface.co/BAAI/bge-m3) que gera um vetor de 1024 dimensao

In [ ]:
#O texto abaixo é a letra de uma música

letra = "Era um cômodo, incômodo, sujo como dragão de komodo  \n  Úmido, eu homem da casa aos seis anos  \n  Mofo no canto, todo, Tv, engodo, pronto pro lodo  \n  Tímido, porra, somos reis, mano  \n  Olhos são eletrodos, sério, topo, trombo corvos  \n  Num cemitério de sonhos, graças a leis, planos  \n  Troco de jogo, vendo roubos, pus a cabeça a prêmio  \n  Ingênuo, colhi sorrisos e falei -- vamos!  \n  É um novo tempo, momento pro novo, ao sabor do vento  \n  Me movo pelo solo onde reinamos  \n  Pondo pontos finais na dor, como doril, anador  \n  Somos a luz do senhor, pode crer, tamo  \n  Construindo, suponho não, creio, meto a mão  \n  Em meio à escuridão, pronto, acertamos  \n  Nosso sorriso sereno hoje é o veneno  \n  Pra quem trouxe tanto ódio pr'onde deitamos  \n    \n  Quem costuma vir de onde eu sou  \n  Às vezes não tem motivos pra  \n  Seguir  \n  Então levanta e anda, vai  \n  Levanta e anda, vai  \n  Levanta e anda  \n  Mas eu sei que vai, que o sonho te traz  \n  Coisas que te faz  \n  Prosseguir  \n  Vai, levanta e anda, vai  \n  Levanta e anda, vai  \n  Levanta e anda, vai  \n  Levanta e anda, vai  \n    \n  Irmão, você não percebeu que você  \n  É o único representante do seu sonho na face da Terra?  \n  Se isso não fizer você correr, chapa  \n  Eu não sei o que vai  \n    \n  Eu sei, sei, cansa  \n  Quem morre ao fim do mês  \n  Nossa grana ou nossa esperança?  \n  Delírio é  \n  Equilíbrio entre nosso martírio e nossa fé  \n  Foi foda contar migalha nos escombros  \n  Lona preta esticada, enxada no ombro e nada vim  \n  Nada enfim, recria  \n  Sozim, com alma cheia de mágoa e as panela vazia  \n  Sonho imundo  \n  Só água na geladeira e eu querendo salvar o mundo  \n  No fundo é tipo David Blaine, mãe assume, pai some  \n  De costume, no máximo é um sobrenome  \n  Sou terror dos clone  \n  Esses boy conhece Marx, nóiz conhece a fome  \n  Então cerre os punhos, sorria  \n  E jamais volte pra sua quebrada de mão e mente vazia  \n    \n  Quem costuma vir de onde eu sou  \n  Às vezes não tem motivos pra  \n  Seguir  \n  Então levanta e anda, vai  \n  Levanta e anda, vai  \n  Levanta e anda  \n  Mas eu sei que vai, que o sonho te traz  \n  Coisas que te faz  \n  Prosseguir  \n  Então levanta e anda, vai  \n  Levanta e anda, vai  \n  Levanta e anda, vai  \n  Levanta e anda  \n    \n  Somos maior, nos basta só  \n  Sonhar, seguir!"


sentencas = letra.split("\n")
sentencas = [sentenca.strip() for sentenca in sentencas]

In [ ]:
# Criando um indice. Neste caso, é necessário configurar o mapeamento


# Criando o conector com elasticsearch
# Modificar a variável PASSWORD para a senha configurado no .env
from elasticsearch import Elasticsearch

HOST = 'http://localhost:9200'
USERNAME = 'elastic'
PASSWORD = 'test10'

client = Elasticsearch(
    HOST,
    basic_auth=(USERNAME,PASSWORD)
)



In [ ]:

mapping = mappings = {
    "properties":{
        "vetor_sentenca":{
            "type":"dense_vector",
            "dims":1024, #dimensao 
            "similarity":"cosine", #algoritmo de similaridade
            "index":True
        },
        "sentenca":{
            "type":"text"
        }
    }
}

In [ ]:
INDEX_LETRA_DENSE = 'vetor_letra_sentenca'

In [ ]:
#criando o indice
if client.indices.exists(index=INDEX_LETRA_DENSE):
    client.indices.delete(index=INDEX_LETRA_DENSE)
client.indices.create(index=INDEX_LETRA_DENSE, body={"mappings":mappings})

In [ ]:
#Para utilizar o modelo de linguagem, é necessário instalar a biblioteca sentence-transformers 
#pip install sentence-transformers
from sentence_transformers import SentenceTransformer   



In [ ]:
#Carregando o modelo de linguagem  - Pode demorar um pouco pois o modelo é grande
model = SentenceTransformer('BAAI/bge-m3')

In [ ]:
#mostrando como o modelo representa uma sentença
sentencas_vetores = list()
for sentenca in sentencas:
    vetor_sentenca = model.encode(sentenca).tolist()  # Convertendo para lista
    doc = {
        "vetor_sentenca": vetor_sentenca,
        "sentenca": sentenca
    }
    sentencas_vetores.append(doc)


In [ ]:
print(f"Mostrando a sentença: {sentencas_vetores[0]['sentenca']} \n")
print(f"Mostrando o vetor correspondente: {sentencas_vetores[0]['vetor_sentenca']}")

In [ ]:
#Inserindo os vetores no índice
for doc in sentencas_vetores:
    client.index(index=INDEX_LETRA_DENSE, body=doc)

In [ ]:
#Pesquisando a sentença mais similar
query_sentenca = "Quem falece no termino"



Essa sentença foi escolhida pois não existe a palavra "falece" ou "termino" e tem mais de um texto com a palavra "quem" 

In [ ]:
#Pesquisa termos semelhantes a query usando o modelo tradicional do elastisearch
query = {
    "query": {
        "match": {
            "sentenca": query_sentenca
        }
    }
}

response = client.search(index=INDEX_LETRA_DENSE, body=query)

In [ ]:
response.body

In [ ]:
#Criando a pesquisa com o modelo de linguagem e pesquisando vetor semelhante
query_vector = model.encode(query_sentenca)

In [ ]:
#Pesquisa de similaridade
query_vector = model.encode(query_sentenca)
query = {
    "query": {
        "knn": {
            "vetor_sentenca": {
                "vector": query_vector,
                "k": 5,  # Número de resultados a serem retornados
                "num_candidates": 100  # Número de candidatos para considerar
            }
        }
    }
}

response = client.search(index=INDEX_LETRA_DENSE, body=query)

In [ ]:
response.body